In [1]:
from keras.models import Model
from keras.optimizers import Adam,SGD
from keras import backend as K
import numpy as np
import rocket_tools as rt
import cv2

Using TensorFlow backend.


In [ ]:
In = Input(shape=(image_size, image_size, 3))

model_unet = rt.model_keras.UNet((image_size,image_size,3),start_ch=16,depth=6,inc_rate=2)

out1 = model_unet(In)
model = Model(inputs = [In], outputs=[out1])

optimzer=Adam(lr=0.001)
model.compile(optimizer=optimzer, loss=[rt.losses.dice_coef_loss],metrics=[rt.losses.dice_coef])


steps=45000
batch_size=32


image_generator = rt.image_gen.ImageGenerator(image_size=(image_size, image_size), snr_mean=6, snr_std=3)
batch_generator = rt.batch_generator.BatchGenerator(image_generator, batch_size,
                                                    n_images=n_images)

print('started training')
for step in range(steps):
    x_batch, y_label = next(batch_generator)

    #x_draw = x_batch[0, :, :, :] * 255
    #y_in_draw = y_label[0, :, :, :] * 255
    #cv2.imshow('x', x_draw.astype(np.uint8))
    #cv2.imshow('y_in', y_in_draw.astype(np.uint8))
    #cv2.waitKey()
    loss = model.train_on_batch(x_batch,y_label)
    # _, summary =
    #sess.run(train_opt_2, feed_dict={x: x_batch, label: y_label})
    # writer.add_summary(summary, step)
    if step % 50 == 0:
        x_batch, y_label = next(batch_generator)
        print(str(step) + ' ' + str(loss))
        x_test = np.zeros((1,128,128,3))
        x_test[0,:,:,:] = x_batch[0,:,:,:]
        res = model.predict(x_test)
        #res = sess.run(output, feed_dict={x: x_test})
        cv2.imwrite('prediction/'+str(step)+'_in.png', res[0, :, :, :] * 255.0)

    if step % 5000 == 0:
        if step!=0:
            model.save_weights('./../saved_models_keras/unet_weight_'+str(step)+'_.h5')

print('finished training')
model.save_weights('./../saved_models_keras/unet_weight_final_.h5')
